# Chaper 2 시계열 데이터의 발견 및 다루기

## 2.2 테이블 집합에서 시계열 데이터 집합 개선하기

In [1]:
import pandas as pd
import io
import requests

In [2]:
YearJoined = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/year_joined.csv")
emails = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/emails.csv")
donations = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/donations.csv")

### groupby 사용법
[설명](https://ponyozzang.tistory.com/291)

In [3]:
YearJoined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


In [4]:
YearJoined.head()

,user,userStats,yearJoined
0,0,silver,2014
1,1,silver,2015
2,2,silver,2016
3,3,bronze,2018
4,4,silver,2018


In [5]:
YearJoined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


이메일에서 null 경우 찾기

In [6]:
emails[emails.emailsOpened < 1]

,emailsOpened,user,week


널값이 없는걸로 보아 이상하므로 한 회원에 대한 기록을 살펴봄

In [7]:
emails[emails.user == 998]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04 00:00:00
25465,3.0,998.0,2017-12-11 00:00:00
25466,3.0,998.0,2017-12-18 00:00:00
25467,3.0,998.0,2018-01-01 00:00:00
25468,3.0,998.0,2018-01-08 00:00:00
25469,2.0,998.0,2018-01-15 00:00:00
25470,3.0,998.0,2018-01-22 00:00:00
25471,2.0,998.0,2018-01-29 00:00:00
25472,3.0,998.0,2018-02-05 00:00:00
25473,3.0,998.0,2018-02-12 00:00:00


일부 주가 누락된 것으로 보임

In [8]:
# ( 마지막 주 - 처음 주)의 일수를 7로 나눈 것
import datetime
date_time_str_max = max(emails[emails.user == 998].week)
date_time_str_min = min(emails[emails.user == 998].week)

date_time_obj_max = datetime.datetime.strptime(date_time_str_max, "%Y-%m-%d %H:%M:%S")
date_time_obj_min = datetime.datetime.strptime(date_time_str_min, "%Y-%m-%d %H:%M:%S")

(date_time_obj_max - date_time_obj_min).days/7

25.0

25.0인걸로 보아 26주간의 기록

해당 직원 주간 데이터 개수 확인

In [9]:
emails[emails.user == 998].shape

(24, 3)

24개 행으로 2개의 주가 누락된것으로 보임

누락된 값을 채워넣기 위해 MultiIndex 사용


In [10]:
complete_idx = pd.MultiIndex.from_product((set(emails.week), set(emails.user)))

In [11]:
# 재색인 메서드 : reindex
# 색인 재설정 메서드 : reset_index

all_email = emails.set_index(['week', 'user']).reindex(complete_idx, fill_value=0).reset_index()
all_email.columns = ['week', 'user', 'emailsOpened']

In [12]:
all_email[all_email.user == 998].sort_values('week')

,week,user,emailsOpened
26410,2015-02-09 00:00:00,998.0,0.0
89473,2015-02-16 00:00:00,998.0,0.0
11857,2015-02-23 00:00:00,998.0,0.0
68991,2015-03-02 00:00:00,998.0,0.0
64679,2015-03-09 00:00:00,998.0,0.0
...,...,...,...
71686,2018-04-30 00:00:00,998.0,3.0
91629,2018-05-07 00:00:00,998.0,3.0
34495,2018-05-14 00:00:00,998.0,3.0
61445,2018-05-21 00:00:00,998.0,3.0


이메일 데이터프레임을 회원별로 그룹화하여 start_date 및 end_date를 결정하고, 이로부터 각 회원이 이메일을 수신해온 주의 시작과 끝을 얻는다.

In [13]:
cutoff_dates = emails.groupby('user').week.agg(['min', 'max']).reset_index()
cutoff_dates = cutoff_dates.reset_index()

In [14]:
cutoff_dates

,index,user,min,max
0,0,1.0,2015-06-29 00:00:00,2018-05-28 00:00:00
1,1,3.0,2018-03-05 00:00:00,2018-04-23 00:00:00
2,2,5.0,2017-06-05 00:00:00,2018-05-28 00:00:00
3,3,6.0,2016-12-05 00:00:00,2018-05-28 00:00:00
4,4,9.0,2016-07-18 00:00:00,2018-05-28 00:00:00
...,...,...,...,...
534,534,991.0,2016-10-24 00:00:00,2016-10-24 00:00:00
535,535,992.0,2015-02-09 00:00:00,2015-07-06 00:00:00
536,536,993.0,2017-09-11 00:00:00,2018-05-28 00:00:00
537,537,995.0,2016-09-05 00:00:00,2018-05-28 00:00:00


쓸모 없는 행 삭제

In [15]:
import warnings
warnings.filterwarnings('ignore')

for _, row in cutoff_dates.iterrows():
    user = row['user']
    start_date = row['min']
    end_date = row['max']
    # start_date 이전의 주에 대한 내용 삭제
    all_email.drop(all_email[all_email.user == user][all_email.week < start_date].index, inplace=True)
    # end_date 이후의 주에 대한 내용 삭제
    all_email.drop(all_email[all_email.user == user][all_email.week > end_date].index, inplace=True)

### 2.2.2 발견된 시계열을 구성하기
* 기부 데이터를 주간 시계열로 바꾼뒤 이메일 데이터와의 비교

In [16]:
donations.timestamp = pd.to_datetime(donations.timestamp)
donations.set_index('timestamp', inplace=True)
agg_don = donations.groupby('user').apply(lambda df: df.amount.resample("W-MON").sum().dropna())

In [17]:
agg_don

user   timestamp 
0.0    2015-03-30      25.0
       2015-04-06       0.0
       2015-04-13       0.0
       2015-04-20       0.0
       2015-04-27       0.0
                      ...  
995.0  2017-09-11       0.0
       2017-09-18       0.0
       2017-09-25       0.0
       2017-10-02    1000.0
998.0  2018-01-08      50.0
Name: amount, Length: 32352, dtype: float64

이메일 및 기부에 대한 데이터프레임 통합

In [18]:
merged_df = pd.DataFrame()

for user, user_email in all_email.groupby('user'):
    # 특정 회원의 기부 데이터 추출
    user_donations = agg_don[agg_don.index.get_level_values('user') == user]

    user_donations = user_donations.droplevel(0)

    user_email = all_email[all_email.user == user]
    user_email.sort_values('week', inplace=True)
    user_email.set_index('week', inplace=True)

    df = pd.merge(user_email, user_donations, how='left', left_index=True, right_index=True)
    df.fillna(0)

    merged_df = merged_df.append(df.reset_index()[['user', 'week', 'emailsOpened', 'amount']])

In [19]:
merged_df.head(5)

,user,week,emailsOpened,amount
0,1.0,2015-06-29 00:00:00,3.0,NaN
1,1.0,2015-07-06 00:00:00,0.0,NaN
2,1.0,2015-07-13 00:00:00,2.0,NaN
3,1.0,2015-07-20 00:00:00,2.0,NaN
4,1.0,2015-07-27 00:00:00,3.0,NaN


데이터를 적절한 주의 수만큼 옮기기

In [20]:
df = merged_df[merged_df.user == 998]
df['target'] = df.amount.shift(1)
df = df.fillna(0)
df.head(7)

,user,week,emailsOpened,amount,target
0,998.0,2017-12-04,1.0,0.0,0.0
1,998.0,2017-12-11,3.0,0.0,0.0
2,998.0,2017-12-18,3.0,0.0,0.0
3,998.0,2017-12-25,0.0,0.0,0.0
4,998.0,2018-01-01,3.0,0.0,0.0
5,998.0,2018-01-08,3.0,50.0,0.0
6,998.0,2018-01-15,2.0,0.0,50.0


## 2.3 타임스탬프의 문제점

### .
    타임스탬프를 볼 떄 가장 먼저 생성과정, 방법, 시기에 대해 생각해봐야함.
    발생 사건에 대한 기록은 실제와 일치하지 않는 경우가 종종 있음.
    위와같은 문제를 피하기 위해서는 데이터의 수집 및 축적에 해당하는 코드를
    읽어보거나, 그 코드를 작성한 사람과 직접 이야기해보거나 타임스탬프의
    생성 절차의 평가에 강능한 한 직접 실행해보아야함.

In [21]:
# 시간차이를 특징으로 사용가능
# 이를 통해 각 사용자의 시간대 추정도 가능
# df['dt'] = df.time - df.time.shift(-1)

### 2.4.3 데이터 평활

판다스에서 다양한 평활 방법을 지원함

지수평활

In [22]:
# 데이터 불러오기
air = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/AirPassengers.csv", names=['Date', 'Passengers'])

In [23]:
air

,Date,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
...,...,...
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390


In [24]:
# 다양한 감쇠요인을 적용하여 평활
air['Smooth.5'] = air.ewm(alpha=0.5).mean().Passengers
air['Smooth.1'] = air.ewm(alpha=0.9).mean().Passengers

alpha 값이 크면 클수록 값의 갱신은 있는 그대로의 현재 값에 가깝도록 더 빨리 갱신

In [25]:
air

,Date,Passengers,Smooth.5,Smooth.1
0,1949-01,112,112.000000,112.000000
1,1949-02,118,116.000000,117.454545
2,1949-03,132,125.142857,130.558559
3,1949-04,129,127.200000,129.155716
4,1949-05,121,124.000000,121.815498
...,...,...,...,...
139,1960-08,606,582.096411,606.665454
140,1960-09,508,545.048205,517.866545
141,1960-10,461,503.024103,466.686655
142,1960-11,390,446.512051,397.668665


## 2.6 시간대

컴퓨터 언어 대부분은 운영체제에 의존하여 시간대 정보를 얻음.<br/>
But, 파이썬에 내장된 자동 시간 검색 함수는 시간대를 인식한 <br/>
타임스탬프를 반환하지않음.

In [34]:
datetime.datetime.utcnow()

datetime.datetime(2021, 12, 14, 18, 28, 37, 359495)

In [35]:
datetime.datetime.now()

datetime.datetime(2021, 12, 14, 18, 28, 37, 999959)

시간대를 부여하지 않았지만, UTC를 반환하지 않았음.

In [36]:
datetime.datetime.now(datetime.timezone.utc)

datetime.datetime(2021, 12, 14, 18, 28, 38, 11841, tzinfo=datetime.timezone.utc)

파이썬으로 시간대 관련 작업을 하기 위해서는 timezone 객체를 생성

In [38]:
# 미 서부에 태평양 표준시 지정
import pytz

western = pytz.timezone('US/Pacific')
western.zone

'US/Pacific'

이후 이 객체를 사용하여 다음과 같이 시간대를 현지화할 수 있음

In [43]:
# API는 시간대를 인지한 시간을 생성하는 두 가지 방법을 제공
# 하나는 'localize'를 이용하는 방식
# 다른 하나는 한 장소의 시간대를 다른 장소의 시간대로 변환하는 방식

# 다음은 현지화하는 방법
loc_dt = western.localize(datetime.datetime(2018, 5, 15, 12, 34, 0))
loc_dt

datetime.datetime(2018, 5, 15, 12, 34, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

But datetime 생성자에 시간대를 직접 전달하면 기대한 결과를 생성하지 않는 경우가 발생가능

In [45]:
london_tz = pytz.timezone('Europe/London')
london_dt = loc_dt.astimezone(london_tz)
london_dt

datetime.datetime(2018, 5, 15, 20, 34, tzinfo=<DstTzInfo 'Europe/London' BST+1:00:00 DST>)

In [47]:
f = '%Y-%m-%d %H:%M:%S %Z%z'
datetime.datetime(2018, 5, 12, 12, 15, 0, tzinfo = london_tz).strftime(f)

# pytz 공식 문서에서 강조하듯 datetime.datetime 생성자의 tzinfo를 사용하면
# London의 예처럼 항상 원하는 결과를 얻게 되는것은 아니다

# pytz 공식 문서에 따르면 이 방법은 일광 절약이 없는 시간대에서
# 원하는 결과를 얻게 해줍니다.

'2018-05-12 12:15:00 LMT-0001'

시간 간격을 계산

In [49]:
# 일반적으로 UTC 형식으로 데이터를 저장한 다음
# 사람이 읽을 수 있는 형태로 출력할 때만 변환
# 시간대에 대하여 날짜 간의 산술 연산도 할 수 있다
event1 = datetime.datetime(2018, 5, 12, 12, 15, 0, tzinfo = london_tz)
event2 = datetime.datetime(2018, 5, 13, 9, 15, 0, tzinfo = western)
event2 - event1
# 위와 같은 계산은 잘못된 시간 간격을 계산
# 서로의 시간대가 적절하게 레이블링되지 않기 때문

datetime.timedelta(days=1, seconds=17520)

In [50]:
event1 = london_tz.localize(datetime.datetime(2018, 5, 12, 12, 15, 0))
event2 = western.localize(datetime.datetime(2018, 5, 13, 9, 15, 0))
event2 - event1

datetime.timedelta(days=1, seconds=18000)

pytz는 국가별 시간대와 일반 시간대의 목록 제공

In [51]:
pytz.common_timezones

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre',
 'Africa/Brazzaville',
 'Africa/Bujumbura',
 'Africa/Cairo',
 'Africa/Casablanca',
 'Africa/Ceuta',
 'Africa/Conakry',
 'Africa/Dakar',
 'Africa/Dar_es_Salaam',
 'Africa/Djibouti',
 'Africa/Douala',
 'Africa/El_Aaiun',
 'Africa/Freetown',
 'Africa/Gaborone',
 'Africa/Harare',
 'Africa/Johannesburg',
 'Africa/Juba',
 'Africa/Kampala',
 'Africa/Khartoum',
 'Africa/Kigali',
 'Africa/Kinshasa',
 'Africa/Lagos',
 'Africa/Libreville',
 'Africa/Lome',
 'Africa/Luanda',
 'Africa/Lubumbashi',
 'Africa/Lusaka',
 'Africa/Malabo',
 'Africa/Maputo',
 'Africa/Maseru',
 'Africa/Mbabane',
 'Africa/Mogadishu',
 'Africa/Monrovia',
 'Africa/Nairobi',
 'Africa/Ndjamena',
 'Africa/Niamey',
 'Africa/Nouakchott',
 'Africa/Ouagadougou',
 'Africa/Porto-Novo',
 'Africa/Sao_Tome',
 'Africa/Tripoli',
 'Africa/Tunis',
 'Africa/Wi

In [52]:
# 국가별 시간대
pytz.country_timezones('RU')

['Europe/Kaliningrad',
 'Europe/Moscow',
 'Europe/Simferopol',
 'Europe/Kirov',
 'Europe/Astrakhan',
 'Europe/Volgograd',
 'Europe/Saratov',
 'Europe/Ulyanovsk',
 'Europe/Samara',
 'Asia/Yekaterinburg',
 'Asia/Omsk',
 'Asia/Novosibirsk',
 'Asia/Barnaul',
 'Asia/Tomsk',
 'Asia/Novokuznetsk',
 'Asia/Krasnoyarsk',
 'Asia/Irkutsk',
 'Asia/Chita',
 'Asia/Yakutsk',
 'Asia/Khandyga',
 'Asia/Vladivostok',
 'Asia/Ust-Nera',
 'Asia/Magadan',
 'Asia/Sakhalin',
 'Asia/Srednekolymsk',
 'Asia/Kamchatka',
 'Asia/Anadyr']

In [54]:
pytz.country_timezones('fr')  # fr <- 프랑스인듯

['Europe/Paris']

일광 절약이 있으면 특정 시간은 두 번 존재 하는 반면(가을이 늦어짐),<br/>
아예 존재하지 않는 시간도 있음(봄을 앞질러 감)

In [59]:
# 시간대
ambig_time = western.localize(datetime.datetime(2002, 10, 27, 1, 30, 00)).astimezone(datetime.timezone.utc)
ambig_time_earlier = ambig_time - datetime.timedelta(hours=1)
ambig_time_later = ambig_time + datetime.timedelta(hours=1)
ambig_time_earlier.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [60]:
ambig_time.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

In [61]:
ambig_time_later.astimezone(western)

datetime.datetime(2002, 10, 27, 2, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

위의 마지막 두 타임스탬프는 동일함. 좋지않은 상황<br/>
이 경에는 is_dst를 사용하서 일광 절약의 적용 여부를 표시해야 합니다.

In [62]:
ambig_time = western.localize(datetime.datetime(2002, 10, 27, 1, 30, 00), is_dst=True).astimezone(datetime.timezone.utc)
ambig_time_earlier = ambig_time - datetime.timedelta(hours=1)
ambig_time_later = ambig_time + datetime.timedelta(hours=1)
ambig_time_earlier.astimezone(western)

datetime.datetime(2002, 10, 27, 0, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [63]:
ambig_time.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [64]:
ambig_time_later.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

이번에는 동일 시간이 두 번 발생하지 않음.<br/>
UTC로부터 오프셋을 확인할 때까지 이러한 방법으로 보여짐.